# Análisis de compras: Groceries store
## Este es un análisis de cesta de compras en una tienda a lo largo de 2 años, primero explorando los datos y encontrando los insights importantes
## después aplicando el algoritmo apriori para encontrar reglas de asociación entre las compras de los productos y dar una respuesta de mercado.
## Data recuperada de: <a href="https://www.kaggle.com/heeraldedhia/groceries-dataset">Link</a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

plt.rcParams['figure.figsize'] = 16, 6

Data = pd.read_csv('../input/groceries-dataset/Groceries_dataset.csv')
Data

In [ ]:
Data.nunique()

In [ ]:
Data['Date'] = pd.to_datetime(Data['Date'])
Data['Year'] = Data['Date'].apply(lambda x : x.year)
Data['Month'] = Data['Date'].apply(lambda x : x.month)
Data['Day'] = Data['Date'].apply(lambda x : x.dayofweek)
DaysOfWeek = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
MonthsOfYear = ['January', 'Febraury', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'Octuber', 'November', 'December']

# Exploración de datos

In [ ]:

f, ax = plt.subplots(1,2,figsize=(30, 6))

Frame = pd.DataFrame(DaysOfWeek)
Compras = []
for index, day in enumerate(DaysOfWeek):
    Filter = Data[Data['Day'] == index]
    Compras.append(len(Filter))

Frame[1] = Compras
Frame = Frame.sort_values(by = 1,ascending=False)

Plot = sns.barplot(x = Frame[0], y = Frame[1], palette='cool', ax=ax[0])
Plot.set_xlabel('Days of week', fontsize=20)
Plot.set_ylabel('Numero de compras', fontsize=20)
Plot.set_title('Distribucion de compras en los dias de la semana', fontsize=30)

Frame = pd.DataFrame(MonthsOfYear)
Compras = []
for index, Month in enumerate(MonthsOfYear):
    Filter = Data[Data['Month'] == index+1]
    Compras.append(len(Filter))

Frame[1] = Compras
Frame = Frame.sort_values(by = 1,ascending=False)

Plot2 = sns.barplot(x = Frame[0], y = Frame[1], palette='rocket', ax=ax[1])
Plot2.set_xlabel('Meses', fontsize=20)
Plot2.set_ylabel('Numero de compras', fontsize=20)
Plot2.set_title('Distribucion de compras en los meses de año', fontsize=30)


In [ ]:
#Comportamiento de compras en los dos años
Frame = Data[Data['Year'] == 2015].Month.value_counts().sort_index(ascending=True)
plt.plot(Frame, c='green', ls='--', marker = 'd', ms=7, label='2015')

Frame = Data[Data['Year'] == 2014].Month.value_counts().sort_index(ascending=True)
plt.plot(Frame, c='orange', ls='-', marker = 'd', ms=7, label='2014')

plt.legend(loc='best', bbox_to_anchor=(1,1))
plt.xticks(list(range(1,13)), MonthsOfYear)
plt.title('Comportamiento de compras de los dos años / meses', fontsize=30)
plt.show()

In [ ]:
Frame = Data[Data['Year'] == 2015].Day.value_counts().sort_index(ascending=True)
plt.plot(Frame, c='blue', ls='-', marker = '*', ms=10, label='2015')

Frame = Data[Data['Year'] == 2014].Day.value_counts().sort_index(ascending=True)
plt.plot(Frame, c='lightblue', ls='--', marker = 'o', ms=10, label='2014')

plt.legend(loc='best', bbox_to_anchor=(1,1))
plt.xticks(list(range(0,7)), DaysOfWeek)
plt.title('Comportamiento de compras de los dos años / semana', fontsize=30)
plt.show()

## Deduciendo rápidamente, nos damos cuenta que la popularidad de la tienda en cuanto a ventas aumento a lo largo de un año, en 2015 el día más popular o más frecuente por los clientes es el miércoles mientras en 2014 es el jueves. claramente nos damos cuenta que los domingos también son días muy populares, esto debido a que la mayoría de las personas descansan los fines de semana y aprovechan el tiempo para ir de compras. No tenemos suficiente información, pero deduciría que los miércoles se colocan más ofertas y eso se refleja en las ventas.

In [ ]:
#Productos destacados y menos populares
f, ax = plt.subplots(3,2,figsize=(30, 18))

def BarPlot(x, y, ax, title, palette):
    Plot = sns.barplot(x = x, y= y, ax=ax, palette=palette)
    Plot.set_title(title, fontsize=30)
    Plot.set_xticklabels(Plot.get_xticklabels(), rotation=20, fontsize=15)

Frame = Data['itemDescription'].value_counts().sort_values(ascending = False)[:10]
BarPlot(Frame.index, Frame.values, ax[0,0], '10 productos mas populares', 'plasma')

Frame = Data['itemDescription'].value_counts().sort_values(ascending = True)[:10]
BarPlot(Frame.index, Frame.values, ax[0,1], '10 productos menos populares', 'rainbow_r')

Frame = Data[(Data['Day'] == 3)]
Frame = Frame['itemDescription'].value_counts().sort_values(ascending=False)[:8]
BarPlot(Frame.index, Frame.values, ax[1,0], 'Productos mas vendidos en jueves', 'Blues')

Frame = Data[(Data['Day'] == 6) | (Data['Day'] == 5) | (Data['Day'] == 4)]
Frame = Frame['itemDescription'].value_counts().sort_values(ascending=False)[:8]
BarPlot(Frame.index, Frame.values, ax[1,1], 'Productos mas vendidos los fines de semana', 'Blues_r')

Frame = Data[(Data['Month'] == 8)]
Frame = Frame['itemDescription'].value_counts().sort_values(ascending=False)[:8]
BarPlot(Frame.index, Frame.values, ax[2,0], 'Productos destacables de agosto', 'Accent')

Frame = Data[(Data['Month'] == 12)]
Frame = Frame['itemDescription'].value_counts().sort_values(ascending=False)[:8]
BarPlot(Frame.index, Frame.values, ax[2,1], 'Productos destacables de diciembre', 'Wistia')

f.tight_layout(pad=3.0)
f.show()


In [ ]:
## Gtrafico de palbras frecuentes
Text = ''
for index, value in enumerate(Data['itemDescription']):
    Text += ' ' + value.replace(" ", '')
Wc = WordCloud().generate(Text)

plt.imshow(Wc, interpolation='bilinear')
plt.axis("off")
plt.show()

## Por mucho, el producto más vendido es leche y vegetables, lo básico en alimentos. Existen productos como utensilios de cocina o productos de preservación. El mercado verdaderamente importante para esta tienda son los productos de canasta básica y frescos: Frutas, vegetales, carnes, leche, entre otros. Lo idea es enfocarse en estos productos e implementar estrategias de venta en estos que la clientela busca.

# Preparación de data para el modelo apriori

In [ ]:
!pip install apyori
from apyori import apriori

OneHot = pd.get_dummies(Data['itemDescription'])
AprioriData = pd.DataFrame(Data)
AprioriData.drop(['itemDescription'], inplace=True, axis=1)
AprioriData = AprioriData.join(OneHot)

Products = Data.itemDescription.unique()
AprioriData = AprioriData.groupby(['Member_number', 'Date'])[Products[:]].sum()
AprioriData = AprioriData.reset_index()
AprioriData.head()

In [ ]:
AprioriData = AprioriData.drop(['Member_number', 'Date'], axis=1)

In [ ]:
AprioriData.head()

In [ ]:
def Names(x):
    for Product in Products:
        if x[Product] != 0:
            x[Product] = Product
    return x

AprioriData = AprioriData.apply(Names, axis=1)
AprioriData.head()

In [ ]:
AprioriArray = AprioriData.values
Transactions = []

for i in range(AprioriArray.shape[0]):
    x = []
    for j in range(AprioriArray.shape[1]):
        if AprioriArray[i,j] != 0:
            x.append(AprioriArray[i,j])
    Transactions.append(x)

Transactions[0:10]

In [ ]:
#Modelo apriori
Rules = apriori(
    transactions = Transactions,
    min_support = 0.0003,
    min_confidence = 0.05,
    min_lift = 3,
    min_length = 2,
)

AssociationRules = list(Rules)

for item in AssociationRules:

    par = item[0] 
    items = [x for x in par]
    print("Regla:       " + items[0] + " -> " + items[1])
    print("Soporte:     " + str(item[1]))
    print("Confianza:   " + str(item[2][0][2]))
    print("Mejora lift: " + str(item[2][0][3]))
    print("***************************************************")

## El algoritmo nos arroja estas reglas, realmente no existe una destacable debido a los bajos valores de soporte.